In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
import random

2023-09-23 13:34:58.467201: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 13:34:58.468812: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 13:34:58.499081: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 13:34:58.499694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 13:34:59.244531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
data = pd.read_csv("data.csv")

data.drop("Unnamed: 0.1", axis=1, inplace = True)
data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [4]:
LOOK_BACK = 30
PREDICT_DAY = 1

In [5]:
def create_dataset(dataset, look_back=LOOK_BACK):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i+look_back, :])
        Y.append(dataset[i+look_back, :])
    return np.array(X), np.array(Y)

X, Y = [],[]
for i in data.columns:
    l = create_dataset(scaled_data)
    X.append(l[0])
    Y.append(l[1])
train_size = int(len(X[0]) * 0.8)

In [6]:
def model(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    X, Y = create_dataset(scaled_data)

In [7]:
X

[array([[[0.82831489, 0.96934454, 0.40381274, ..., 0.62249428,
          0.73698041, 0.75533808],
         [0.82922184, 0.96820168, 0.40606658, ..., 0.62061079,
          0.72845995, 0.75289146],
         [0.8333938 , 0.96833613, 0.40597267, ..., 0.61509485,
          0.73682115, 0.75155694],
         ...,
         [0.80509704, 0.93472269, 0.43773771, ..., 0.62249428,
          0.72304507, 0.75911922],
         [0.80201342, 0.93815126, 0.43022491, ..., 0.62034172,
          0.71978022, 0.76067616],
         [0.79974605, 0.93492437, 0.43449782, ..., 0.61845823,
          0.72551362, 0.7555605 ]],
 
        [[0.82922184, 0.96820168, 0.40606658, ..., 0.62061079,
          0.72845995, 0.75289146],
         [0.8333938 , 0.96833613, 0.40597267, ..., 0.61509485,
          0.73682115, 0.75155694],
         [0.83148921, 0.96228571, 0.40871954, ..., 0.61442217,
          0.74303233, 0.74911032],
         ...,
         [0.80201342, 0.93815126, 0.43022491, ..., 0.62034172,
          0.71978022, 0.

In [8]:
data.shape

(3648, 10)

In [53]:
model = Sequential()
model.add(
    Dense(32,input_shape=(LOOK_BACK, data.shape[1]), activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu"))

model.add(Dense(data.shape[1]))

model.compile(loss="mean_squared_error", optimizer="adam")

In [54]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 30, 32)            384       
                                                                 
 dense_46 (Dense)            (None, 30, 64)            2112      
                                                                 
 dropout_7 (Dropout)         (None, 30, 64)            0         
                                                                 
 dense_47 (Dense)            (None, 30, 128)           8320      
                                                                 
 dropout_8 (Dropout)         (None, 30, 128)           0         
                                                                 
 dense_48 (Dense)            (None, 30, 64)            8256      
                                                                 
 dropout_9 (Dropout)         (None, 30, 64)          

In [55]:
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
2894/2894 [==============================] - 3s 874us/step - loss: 0.1490
Epoch 2/10
2894/2894 [==============================] - 2s 840us/step - loss: 0.1371
Epoch 3/10
2894/2894 [==============================] - 2s 863us/step - loss: 0.1360
Epoch 4/10
2894/2894 [==============================] - 3s 876us/step - loss: 0.1349
Epoch 5/10
2894/2894 [==============================] - 3s 889us/step - loss: 0.1340
Epoch 6/10
2894/2894 [==============================] - 2s 852us/step - loss: 0.1342
Epoch 7/10
2894/2894 [==============================] - 3s 882us/step - loss: 0.1348
Epoch 8/10
2894/2894 [==============================] - 3s 938us/step - loss: 0.1345
Epoch 9/10
2894/2894 [==============================] - 3s 1ms/step - loss: 0.1342
Epoch 10/10
2894/2894 [==============================] - 6s 2ms/step - loss: 0.1337


In [56]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainY_original = scaler.inverse_transform(trainY)
testY_original = scaler.inverse_transform(testY)

23/23 [==============================] - 0s 2ms/step


In [57]:
testPredictions = []
for i in range(len(testX)):
    prediction = model.predict(testX[i:i+1])
    testPredictions.append(prediction[0])

testPredictions = np.array(testPredictions)


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


In [93]:
random_start_index = random.randint(0, len(X) - PREDICT_DAY)


random_input_sequence = X[random_start_index]
true_output = Y[random_start_index]


random_input_reshaped = np.reshape(random_input_sequence, (1, LOOK_BACK, data.shape[1]))

predicted_output = model.predict(random_input_reshaped)
print(predicted_output[0].transpose().shape)
print(true_output.shape)
predicted_output = predicted_output[0].transpose()
MSE = 0
MAE = 0
MAPE = 0
for i in range(30):
    l = true_output+predicted_output[:,i]
    MSE += np.sum(np.square(l))/11
    m = np.abs(true_output-predicted_output[:,i])
    MAE += np.mean(m)
    n = (m/true_output)
    MAPE += np.mean(n)
MSE /= 30
MAE /= 30
MAPE /= 30

print('Random Sequence Evaluation:')
print('MSE:', MSE)
print('MAE:', MAE)
print('MAPE:', MAPE)


1/1 [==============================] - 0s 15ms/step
(11, 30)
(11,)
Random Sequence Evaluation:
MSE: 1.586263733541835
MAE: 0.20259009094693045
MAPE: 0.2776675232669042
